### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности.

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [38]:
!pip install razdel datasets seqeval
!pip install -U accelerate
!pip install -U transformers
!pip install pymorphy2

In [39]:
import pandas as pd
from razdel import tokenize

def get_token(item):
  raw_toks = list(tokenize(item['video_info']))
  words = [tok.text for tok in raw_toks]
  return {'tokens': words}


In [40]:
input_data = pd.read_csv("submission_sample.csv")
input_data = pd.DataFrame([get_token(item) for i, item in input_data.iterrows()])
input_data

,tokens
0,"[<, НАЗВАНИЕ, :, >, Антон, Сунцов, =, отзыв, в..."
1,"[<, НАЗВАНИЕ, :, >, Лучшее, в, матче, Белогорь..."
2,"[<, НАЗВАНИЕ, :, >, ТОП, 5, камерных, психолог..."
3,"[<, НАЗВАНИЕ, :, >, Очень, странные, дела, ,, ..."
4,"[<, НАЗВАНИЕ, :, >, Это, поражение, ,, но, шан..."
...,...
95,"[<, НАЗВАНИЕ, :, >, Бой, №, 06, ., Турнир, Лег..."
96,"[<, НАЗВАНИЕ, :, >, Ошибки, детективов, могут,..."
97,"[<, НАЗВАНИЕ, :, >, 29, июля, 2022, г, ., Кучу..."
98,"[<, НАЗВАНИЕ, :, >, Как, спаять, лопнувший, пл..."


### Предобработка текста

In [41]:
import nltk
from nltk.corpus import stopwords
import pymorphy2
import re
from sklearn.model_selection import train_test_split
import numpy as np

nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
def prepare_learning_data(dataset):
  tokens = []
  tags = []
  ids = []
  for i, item in dataset.iterrows():
    new_token = []
    new_tag = []
    new_id = []

    for id, row in enumerate(zip(item['tokens'], item['tags'])):
      token = row[0]
      tag = row[1]
      if token not in stopwords_ru and re.match(r'^[А-Яа-яa-zA-Z0-9:’]+$', token) is not None:
        new_token.append(morph.normal_forms(token.lower())[0])
        new_tag.append(tag)
        new_id.append(id)

    tokens.append(new_token)
    tags.append(new_tag)
    ids.append(new_id)

  return pd.DataFrame({'tokens':tokens, 'tags':tags, 'ids':ids})

def prepare_test_data(dataset):
  tokens = []
  ids = []
  lens=[]
  for i, item in dataset.iterrows():
    new_token = []
    new_id = []

    for id, token in enumerate(item['tokens']):
      if (token not in stopwords_ru and re.match(r'^[А-Яа-яa-zA-Z0-9:’]+$', token) is not None):
        new_token.append(morph.normal_forms(token.lower())[0])
        new_id.append(id)

    tokens.append(new_token)
    ids.append(new_id)
    lens.append(len(item['tokens']))

  return pd.DataFrame({'tokens':tokens, 'ids':ids, 'lens':lens})


def get_old(dataset):
  olds = []
  for _, item in dataset.iterrows():
    old = ['O'] * (item['lens'])
    for i in range(len(item['ids'])):
      old[item['ids'][i]] = item['tags'][i]
    olds.append(old)
  return pd.DataFrame({'tags':olds})

In [43]:
input_fixed_data = prepare_test_data(input_data)

from datasets import Dataset, DatasetDict

input_dataset = DatasetDict({
        'test': Dataset.from_pandas(pd.DataFrame({"tokens": input_fixed_data['tokens']}))
})

# input_dataset
input_fixed_data

,tokens,ids,lens
0,"[название, :, антон, сунец, отзыв, выпускник, ...","[1, 2, 4, 5, 7, 8, 10, 11, 13, 16, 17, 19, 20,...",51
1,"[название, :, хороший, матч, белогорье, локомо...","[1, 2, 4, 6, 7, 9, 11, 12, 13, 14, 15, 16, 18,...",153
2,"[название, :, топ, 5, камерный, психологически...","[1, 2, 4, 5, 6, 7, 8, 10, 11, 13, 15, 16, 18, ...",140
3,"[название, :, очень, странный, дело, 2, сезон,...","[1, 2, 4, 5, 6, 8, 9, 11, 12, 13, 14, 16, 17, ...",112
4,"[название, :, это, поражение, шанс, :, линдерм...","[1, 2, 4, 5, 8, 11, 12, 14, 15, 17, 19, 21, 22...",69
...,...,...,...
95,"[название, :, бой, 06, турнир, легион, кудо, о...","[1, 2, 4, 6, 8, 9, 10, 12, 13, 16, 18, 20, 22,...",76
96,"[название, :, ошибка, детектив, мочь, стать, п...","[1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 1...",55
97,"[название, :, 29, июль, 2022, г, кучугур, стра...","[1, 2, 4, 5, 6, 7, 9, 11, 12, 14, 15, 17, 18, ...",59
98,"[название, :, как, спаять, лопнуть, пластиковы...","[1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 1...",179


### Инициализируем модель

In [44]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_list = ['O',
              'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

model = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/ner_bert.bin', num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/ner_bert.bin', device='gpu')

In [45]:
def bert_token(dataset):
  return tokenizer(dataset["tokens"], truncation=True, is_split_into_words=True)


tokenized_datasets = input_dataset.map(bert_token, batched=True)
trainer = Trainer(model, tokenizer=tokenizer)

predictions = trainer.predict(tokenized_datasets["test"])
predictions1 = np.argmax(predictions[0], axis=2)
true_predictions = [[label_list[p] for p in row] for row in predictions1]
output_data =get_old(pd.DataFrame({'tags':true_predictions, 'ids':input_fixed_data['ids'], 'lens':input_fixed_data['lens']}))
output_data

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,tags
0,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[O, O, O, O, O, B-название проекта, I-название..."
3,"[O, O, O, O, O, B-название проекта, I-название..."
4,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...
95,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
96,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
97,"[O, O, O, O, O, B-Дата, I-Дата, I-Дата, O, I-Д..."
98,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [46]:
month = ['января', 'января', 'февраль', 'февраль', 'март', 'марта', 'апрель', 'апреля', 'май', 'мая', 'июнь', 'июня',
         'июль', 'июля', 'август', 'августа', 'сентябрь', 'сентября', 'октябрь', 'октября', 'ноябрь', 'ноября', 'декабрь', 'декабря']

In [47]:
new_data = pd.DataFrame({'video_info': input_data['tokens'], 'entities_prediction': output_data['tags']})

for _, i in new_data.iterrows():
  row = i['video_info']
  label_list = i['entities_prediction']
  for word in range(len(row)-2):
    if (row[word].isdigit() and re.fullmatch('год.', row[word+1].lower()) is not None):
      label_list[word] = 'B-data'
      label_list[word+1] = 'I-data'
    elif (row[word].isdigit() and row[word+1].lower() in month):
      label_list[word] = 'B-data'
      label_list[word+1] = 'I-data'
      if row[word+2].isdigit():
        label_list[word+2] = 'I-data'
    elif (row[word].isdigit() and re.fullmatch('сезо.', row[word+1].lower()) is not None):
      label_list[word] = 'B-сезон'
      label_list[word+1] = 'I-сезон'
  i['entities_prediction'] = label_list


In [48]:
new_data

,video_info,entities_prediction
0,"[<, НАЗВАНИЕ, :, >, Антон, Сунцов, =, отзыв, в...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[<, НАЗВАНИЕ, :, >, Лучшее, в, матче, Белогорь...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[<, НАЗВАНИЕ, :, >, ТОП, 5, камерных, психолог...","[O, O, O, O, O, B-название проекта, I-название..."
3,"[<, НАЗВАНИЕ, :, >, Очень, странные, дела, ,, ...","[O, O, O, O, O, B-название проекта, I-название..."
4,"[<, НАЗВАНИЕ, :, >, Это, поражение, ,, но, шан...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
95,"[<, НАЗВАНИЕ, :, >, Бой, №, 06, ., Турнир, Лег...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
96,"[<, НАЗВАНИЕ, :, >, Ошибки, детективов, могут,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
97,"[<, НАЗВАНИЕ, :, >, 29, июля, 2022, г, ., Кучу...","[O, O, O, O, B-data, I-data, I-data, I-Дата, O..."
98,"[<, НАЗВАНИЕ, :, >, Как, спаять, лопнувший, пл...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [49]:
new_data.to_csv('submission.csv')